In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt

import settings as s
import Hearis as h
from Custom import Mouse

import scipy.io as sio

batch = s.Batch()

water_path = '/home/pouple/PhD/Code/MFB_Paper/Water'
mfb_path = '/home/pouple/PhD/Code/MFB_Paper/MFB'

plt.style.use('fivethirtyeight')

In [2]:
class MouseMFB(object):
    def __init__(self, path):
        self.path = path
        data = self.__load_data(self.path)
        self.day = data['Day']
        self.corr = data['Correct']
        self.idsound = data['SoundId']
        self.idtrial = data['TrialId']
        self.rule = data['Rule'][0]
        
        self.split_in_sessions()
        
        print('Mouse', os.path.basename(self.path)[1:-4], 'Loaded')
    
    def __load_data(self, path):
        return sio.loadmat(path)
    
    def split_in_sessions(self):
        self.corrs = []
        self.idsounds = []
        self.idtrials = []
        for day in np.unique(self.day):
            indices = np.where(self.day == day)
            self.corrs.append(self.corr[indices])
            self.idsounds.append(self.idsound[indices])
            self.idtrials.append(self.idtrial[indices])
    
    def get_session_score(self):
        scores = [np.sum(corr)*100/len(corr) for corr in self.corrs]
        return scores
    
    def get_best_sessions_scores(self, n_sess):
        best_sess = np.argsort(self.get_session_score())[-n_sess:]
        return best_sess
    def keep_best_sessions(self, n_sess):
        best_sess = self.get_best_sessions_scores(n_sess)
        self.best_corrs = list(np.array(self.corrs, dtype=object)[best_sess])
        self.best_idsounds = list(np.array(self.idsounds, dtype=object)[best_sess])
        self.best_idtrials = list(np.array(self.idtrials, dtype=object)[best_sess])
            
        
            
            
        

In [3]:
# Paths for MFB data
# Paths for Water Data
def load_water():
    mice_id = batch.id_first_batch
    mice_water = [Mouse(path=os.path.join(water_path, f), tag=['PC'], collab=False, rmgaps=False) for f in mice_id]
    
    return mice_water 

def load_MFB():
    # Load MFB mice data
    path = '/home/pouple/PhD/Code/MFB_Paper/MFB/Psychophysics_MFB'
    mice_mfb = os.listdir(path)
    paths = [os.path.join(path, f) for f in os.listdir(path)]
    mice_mfb = [MouseMFB(path) for path in paths]
    
    return mice_mfb

In [ ]:
mice_water = load_water()
mice_mfb = load_MFB()

Processing files for mice 660459...
PC_08022021_660459_1.DAT
PC_05022021_660459_1.DAT
PC_11022021_660459_1.DAT
PC_01022021_660459_1.DAT
PC_12022021_660459_1.DAT
PC_28012021_660459_1.DAT
PC_03022021_660459_1.DAT
PC_02022021_660459_1.DAT
PC_29012021_660459_1.DAT
PC_04022021_660459_1.DAT
PC_10022021_660459_1.DAT
PC_09022021_660459_1.DAT
PC_27012021_660459_1.DAT
Processing files for mice 660461...
PC_01022021_660461_1.DAT
PC_03022021_660461_1.DAT
PC_12022021_660461_1.DAT
PC_18022021_660461_1.DAT
PC_23022021_660461_1.DAT
PC_11022021_660461_1.DAT
PC_09022021_660461_1.DAT
PC_02022021_660461_1.DAT
PC_10022021_660461_1.DAT
PC_16022021_660461_1.DAT
PC_17022021_660461_1.DAT
PC_22022021_660461_1.DAT
PC_04022021_660461_1.DAT
PC_19022021_660461_1.DAT
PC_08022021_660461_1.DAT
PC_05022021_660461_1.DAT
PC_25022021_660461_1.DAT
PC_26022021_660461_1.DAT
Processing files for mice 660462...
PC_08022021_660462_1.DAT
PC_11022021_660462_1.DAT
PC_04022021_660462_1.DAT
PC_12022021_660462_2.DAT
PC_01022021_66046

In [ ]:
# Select only best performing sessions
for mouse in mice_water: mouse.select_best_trials(3)
for mouse in mice_mfb: mouse.keep_best_sessions(3)

### Observation 
There seems to be a log preference shape whern looking at individual mouse in the no go section. Represents the shift around threshold. Maybe good to quantify. PS for MFB seems to have less variance

In [ ]:
# Peak performance : bar plot go vs nogo sur les 2/3 meilleures sessions + la courbe PS

scores_water = np.array([mouse.score_by_task(plot=False) for mouse in mice_water])
mean_scores_water = np.mean(scores_water, axis=0)

scores_mfb = np.empty((len(mice_mfb), 16))
for i, mouse in enumerate(mice_mfb):
    all_taskid = np.concatenate(mouse.best_idsounds)
    all_corr = np.concatenate(mouse.best_corrs)
    
    for t in np.sort(np.unique(all_taskid)):
        idxs = np.where(all_taskid == t)[0]
        curr_corr = all_corr[idxs]
        scores_mfb[i] = (np.sum(curr_corr)*100/len(curr_corr))
mean_scores_mfb = np.mean(scores_mfb, axis=0)

# Psycoacoustics 
probs = np.array([mouse.psychoacoustic(plot=False)[1] for mouse in mice_water])
reverse = [mouse.reversed for mouse in mice_water]
probs = [np.abs(1-p) if reverse[i] else p for i, p in enumerate(probs)]

######## THIS IS NOT WORKING, RECALCULATE PSYCOCURVE
psycos_mfb = np.empty((len(mice_mfb), 16))
for i, mouse in enumerate(mice_mfb):
    all_trailsid = np.concatenate(mouse.best_idtrials)
    all_corr = np.concatenate(mouse.best_corrs)
    all_taskid = np.concatenate(mouse.best_idsounds)
    
    psycho = np.array([c if all_trailsid[i] == 1 else not(c) for i, c in enumerate(all_corr)]) 
    
    scores = np.empty((16,))
    for j, t in enumerate(np.sort(np.unique(all_taskid))):
        idxs = np.where(all_taskid == t)[0]
        curr_corr = psycho[idxs]
        score = np.sum(curr_corr)*100/len(curr_corr)
        scores[j] = score

    psycos_mfb[i] = scores
psycos_mfb = [np.abs(100 - s) if mice_mfb[i].rule == 'H' else s for i, s in enumerate(psycos_mfb)]    
psycos_mfb = np.array(psycos_mfb)      
        
fig, axs = plt.subplots(1, 2, figsize=(4, 6))

axs[0].bar([1, 2], [np.mean(mean_scores_water[:8]), np.mean(mean_scores_water[8:])])
axs[0].set_xticks([1, 2])
axs[0].set_xticklabels(['Go', 'NoGo'])
axs[0].set_yticks(np.linspace(0, 90, 10))
axs[0].set_title('Water')

axs[1].bar([1, 2], [np.mean(mean_scores_mfb[:8]), np.mean(mean_scores_mfb[8:])])
axs[1].set_xticks([1, 2])
axs[1].set_xticklabels(['Go', 'NoGo'])
axs[1].set_yticks(np.linspace(0, 90, 10))
axs[1].set_title('MFB')

plt.show()
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

scale = np.geomspace(6e3, 16e3, 16)
for p in probs:
    axs[0].plot(scale, p, c='gray')
axs[0].plot(scale, np.mean(probs, axis=0), c='blue')
axs[0].set_title('Water')


for p in psycos_mfb:
    axs[1].plot(scale, p/100, c='gray')
axs[1].plot(scale, np.mean(psycos_mfb/100, axis=0), c='blue')
axs[1].set_title('MFB')

plt.show()


## MISSING REMOVE BLOCKS WITH NO RESPONSES


# Learning curves
Accuracy depending on days or trials. Sliding window with smooth parameter for trials 


In [ ]:
mice_water = load_water()
mice_mfb = load_MFB()

In [ ]:
from datetime import date

scores_water = [mouse.perf() for mouse in mice_water]
scores_mfb = [[np.sum(c)*100/len(c) for c in mouse.corrs] for mouse in mice_mfb]


dates = []
for i, m in enumerate(scores_water):
    dates.append([(d - m[1][0]).astype('timedelta64[D]') + 1 for d in m[1:]])
scores_water = [np.array(s[0]) for s in scores_water]


    

fig, axs = plt.subplots(2, figsize=(15, 10))
for i, s in enumerate(scores_water):
    #axs[0].plot(dates[i][0], s, color='grey', marker='o', markersize=8)
    axs[0].plot(s, color='grey', marker='o', markersize=8)
axs[0].set_title('Water')
axs[0].set_ylim((40, 100))
axs[0].set_xlim((0, 30))


for i, s in enumerate(scores_mfb):
    axs[1].plot(s, color='grey', marker='o', markersize=8)
axs[1].set_title('MFB')
axs[1].set_ylim((40, 100))
axs[1].set_xlim((0, 30))

    

